In [ ]:
# =========================
# Imports & Paths
# =========================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

sns.set(style="whitegrid", rc={"figure.dpi": 140})

# Project paths
BASE = os.path.abspath(os.path.join(os.path.dirname(__file__), ".."))
RAW_DIR = os.path.join(BASE, "data", "raw")
CSV_DIR = os.path.join(BASE, "result", "csv")
FIG_DIR = os.path.join(BASE, "result", "figures")

os.makedirs(FIG_DIR, exist_ok=True)

# Sharpe ratio function
def sharpe_ratio(returns, rf=0.0):
    excess = returns - rf/252
    return np.sqrt(252) * excess.mean() / (returns.std() + 1e-12)


In [ ]:
# =========================
# Load price data
# =========================
def load_price(path, col_name):
    df = pd.read_csv(path, parse_dates=["Date"])
    df = df.sort_values("Date")
    return df[["Date", "Close"]].rename(columns={"Close": col_name})

tsla = load_price(os.path.join(RAW_DIR, "TSLA_raw.csv"), "TSLA")
bnd  = load_price(os.path.join(RAW_DIR, "BND_raw.csv"), "BND")
spy  = load_price(os.path.join(RAW_DIR, "SPY_raw.csv"), "SPY")

prices = tsla.merge(bnd, on="Date").merge(spy, on="Date")


In [ ]:
# =========================
# Backtest period
# =========================
backtest_start = prices["Date"].max() - pd.DateOffset(years=1)
prices_bt = prices[prices["Date"] >= backtest_start].reset_index(drop=True)

print(f" Backtest from {prices_bt['Date'].min().date()} to {prices_bt['Date'].max().date()}")


In [ ]:
# =========================
# Load optimized weights
# =========================
summary_path = os.path.join(CSV_DIR, "portfolio_optimization_summary.csv")
summary_df = pd.read_csv(summary_path)

# Choose portfolio: "Max Sharpe" or "Min Volatility"
portfolio_choice = "Max Sharpe"
weights_row = summary_df.set_index("Portfolio").loc[portfolio_choice]
weights = np.array([weights_row["TSLA_w"], weights_row["BND_w"], weights_row["SPY_w"]])

print(f"Using {portfolio_choice} weights:", weights)


In [ ]:
# =========================
# Daily returns
# =========================
rets_bt = prices_bt.set_index("Date").pct_change().dropna()

# Strategy portfolio daily returns
strategy_daily = (rets_bt * weights).sum(axis=1)

# Benchmark: 60% SPY / 40% BND
bm_weights = np.array([0.0, 0.4, 0.6])
benchmark_daily = (rets_bt * bm_weights).sum(axis=1)


In [ ]:
# =========================
# Cumulative returns plot
# =========================
cum_strategy = (1 + strategy_daily).cumprod()
cum_benchmark = (1 + benchmark_daily).cumprod()

plt.figure(figsize=(10,6))
plt.plot(cum_strategy, label=f"Strategy ({portfolio_choice})")
plt.plot(cum_benchmark, label="Benchmark (60% SPY / 40% BND)")
plt.title("Portfolio Backtest — Last 1 Year")
plt.xlabel("Date"); plt.ylabel("Cumulative Return")
plt.legend()
plt.savefig(os.path.join(FIG_DIR, "backtest_strategy_vs_benchmark.png"), dpi=160)
plt.show()


In [ ]:
# =========================
# Performance metrics
# =========================
metrics = pd.DataFrame({
    "Portfolio": [portfolio_choice, "Benchmark 60/40"],
    "Total Return": [cum_strategy.iloc[-1] - 1, cum_benchmark.iloc[-1] - 1],
    "Sharpe Ratio": [sharpe_ratio(strategy_daily), sharpe_ratio(benchmark_daily)]
})

metrics_path = os.path.join(CSV_DIR, "backtest_metrics.csv")
metrics.to_csv(metrics_path, index=False)
metrics


In [ ]:
# =========================
# 🏁 CELL 8 — Verdict
# =========================
str_ret = float(cum_strategy.iloc[-1] - 1)
bm_ret = float(cum_benchmark.iloc[-1] - 1)
str_sharpe = sharpe_ratio(strategy_daily)
bm_sharpe = sharpe_ratio(benchmark_daily)

print("BACKTEST SUMMARY")
print("-------------------")
print(f"Strategy  → Total Return: {str_ret*100:.2f}%, Sharpe: {str_sharpe:.3f}")
print(f"Benchmark → Total Return: {bm_ret*100:.2f}%, Sharpe: {bm_sharpe:.3f}")

if str_ret > bm_ret and str_sharpe > bm_sharpe:
    print("\nThe strategy outperformed the benchmark in both return and risk-adjusted performance.")
elif str_ret > bm_ret:
    print("\n The strategy had higher returns but lower Sharpe ratio than the benchmark.")
elif str_sharpe > bm_sharpe:
    print("\n The strategy had a better Sharpe ratio but lower total returns.")
else:
    print("\n The strategy underperformed the benchmark in both return and risk-adjusted terms.")

print("\n📄 Metrics saved to:", metrics_path)
print("🖼  Plot saved to:", os.path.join(FIG_DIR, "backtest_strategy_vs_benchmark.png"))
